In [1]:
import articles.article_fetch as article_fetch
import articles.articles_info as article_info
from articles import article

import csv
import codecs

import pandas as pd
from classifiers.random_forest import RandomForest
from classifiers.svm_classifier import SVMClassifier
from classifiers.mlp_classifier import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_recall_fscore_support

import clinical_trials.clinical_trial_fetch as ct_fetch
import clinical_trials.clinical_trials_info as ct_info
from clinical_trials import clinical_trial

import csv
import csv_data.csv as csv_data

from dataframe import create_dataframe
from dataframe import calculate_attributes

from common_functions import common_functions

import pandas as pd

import importlib

# I get the gold standard
gold_standard = pd.read_csv('ClinicalPmidsALL.csv', encoding = 'ISO-8859-1', sep = ';')

# Now I change the common answer from string to numerical
csv_data.numerical_answers(gold_standard)

# Let's correct data on csv (common answer sometimes is not correct)
csv_data.correctData(gold_standard)

# I get all the clinical trials ID and I get the articles in xml
Clinical_trials = ct_fetch.get_xml_doms(gold_standard['CT'].tolist())

# I get all the PMID and I get the articles in xml
PubMed_id_string = list(map(str, gold_standard['PMID'].tolist())) # I get the PMID as list of strings
PubMed_articles = article_fetch.fetch_many_articles(PubMed_id_string, local=True) # It takes time to fetch the articles

# Let's map every clinical trial to every article (and remove the one that are not present) and common answer
df = create_dataframe.get_base_dataframe(gold_standard, Clinical_trials, PubMed_articles)

# standard, spacy or stanford
ct_org_sample = 'standard'
ar_org_sample = 'spacy'

C:\Users\Brescia\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


article 27953647  not found.
article 27948541  not found.
article 27943881  not found.
article 27949797  not found.
article 27955116  not found.
article 27950623  not found.
article 27945102  not found.
clinical trial NCT02659670  not found.
article 27198327  not found.


In [2]:
#lam = gold_standard['CT']
#lam.index[lam == 'NCT00003204'].tolist()[0]

ct_last_names, ct_first_name_initials, ct_first_names = ct_info.get_all_name_parts(df['CT'].tolist(), gold_standard)

df['ct_last_name'], df['ct_first_name_initial'], df['ct_first_name'] = [ct_last_names, ct_first_name_initials, ct_first_names]

In [3]:
# print(sum(x is not None for x in ct_last_names))
df = df.dropna().reset_index(drop = True)

In [4]:
ar_last_names, ar_first_name_initials, ar_first_names = article_info.get_all_name_parts(df['PubMed'].tolist(), gold_standard)

df['ar_last_name'], df['ar_first_name_initial'], df['ar_first_name'] = [ar_last_names, ar_first_name_initials, ar_first_names]
df.dropna().reset_index(drop = True)
print()

In [5]:
# now that we have the base information, let's add other attributes

# we get the organization of the principal investigator, so we don't need to pass his/her name
ct_organization_names = ct_info.get_all_organization_names(df['CT'].tolist(), df['ct_last_name'].tolist(),
                                                           df['ct_first_name_initial'].tolist(),
                                                           sample=ct_org_sample)

# Now we insert it in the dataframe
if('ct_organization' not in df):
    df.insert(6, 'ct_organization', ct_organization_names)
    

# I get the e-mails
ct_mails = ct_info.get_all_mails(df['CT'].tolist(), df['ct_last_name'].tolist(), df['ct_first_name_initial'].tolist())

if('ct_mail' not in df):
    df.insert(7, 'ct_mail', ct_mails)
    
# I get the year
ct_years = ct_info.get_all_years(df['CT'].tolist())
    
if('ct_year' not in df):
    df.insert(8, 'ct_year', ct_years)
    
# I get the initials of the name
ct_initials = ct_info.get_all_initials(df['ct_first_name'].tolist())

if('ct_initials' not in df):
    df.insert(9, 'ct_initials', ct_initials)
    
# I get the title
ct_titles = ct_info.get_all_titles(df['CT'].tolist())

if('ct_title' not in df):
    df.insert(10, 'ct_title', ct_titles)
    
# I get the country and the city
ct_countries, ct_cities = ct_info.get_all_countries_and_cities(df['CT'].tolist())

if('ct_country' not in df):
    df.insert(11, 'ct_country', ct_countries)
    
if('ct_city' not in df):
    df.insert(12, 'ct_city', ct_cities)

NCT00730210 doesn't have an organization.
NCT02220283 doesn't have an organization.


In [6]:
print(sum(x is not None for x in df['ct_mail']))

267


In [7]:
# Let's add the same attributes for the articles

ar_organization_names, ar_locations = article_info.get_all_organizations_locations(df['PubMed'].tolist(),
                                                           df['ar_last_name'].tolist(),
                                                           df['ar_first_name_initial'].tolist(),
                                                           sample = ar_org_sample)
df['ar_organization'] = ar_organization_names

ar_mails = article_info.get_all_mails(df['PubMed'].tolist(), df['ar_last_name'].tolist(),
                                      df['ar_first_name_initial'].tolist())
df['ar_mail'] = ar_mails

ar_years = article_info.get_all_years(df['PubMed'].tolist())

df['ar_year'] = ar_years

ar_initials = article_info.get_all_initials(df['ar_first_name'])

df['ar_initials'] = ar_initials

ar_titles = article_info.get_all_titles(df['PubMed'].tolist())

df['ar_title'] = ar_titles
df['ar_location'] = ar_locations

print(sum(x is not None for x in ar_organization_names))
print(sum(x is not None for x in ar_mails))

475
126


In [8]:
# I now calculate useful attributes for the classifiers
first_name_equalities = calculate_attributes.get_string_arrays_similarity(df['ct_first_name'].tolist(),
                                                                          df['ar_first_name'].tolist())
organization_similarities, organization_type_equalities = calculate_attributes.get_organization_similarity_and_type_equality(df['ct_organization'].tolist(),
                                                                             df['ar_organization'].tolist(),
                                                                             ct_org_sample, ar_org_sample)
email_equalities = calculate_attributes.get_arrays_equality(df['ct_mail'].tolist(), df['ar_mail'].tolist())
year_differences = calculate_attributes.get_year_differences(df['ct_year'].tolist(), df['ar_year'].tolist())
last_name_lengths = calculate_attributes.get_last_name_lengths(df['ct_last_name'].tolist())
initials_equality = calculate_attributes.get_arrays_equality(df['ct_initials'].tolist(), df['ar_initials'].tolist())
namespace_sizes = calculate_attributes.get_namespace_ambiguities(df['ct_last_name'].tolist(),
                                                                 df['ct_first_name_initial'].tolist())
country_equalities, city_equalities = calculate_attributes.get_location_equalities(df['ct_country'].tolist(),
                                                                                   df['ct_city'].tolist(),
                                                                                   df['ar_location'].tolist())

# Let's add the attributes to the data frame
df['first_name_equality'], df['organization_similarity'] = [first_name_equalities, organization_similarities]
df['email_equality'], df['year_difference'] = [email_equalities, year_differences]
df['last_name_length'], df['initials_equality'] = [last_name_lengths, initials_equality]
df['namespace_size'], df ['country_equality'] = [namespace_sizes, country_equalities]
df['city_equality'], df['organization_type_equality'] = [city_equalities, organization_type_equalities]

In [9]:
from java_libraries import java_server

clinical_trials = df['CT'].tolist()
articles = df['PubMed'].tolist()

server = java_server.JavaServer()

jds = []
sts = []
for i in range(len(articles)):
    ct_texts = clinical_trials[i].get_text()
    ct_texts.append(clinical_trials[i].get_title())
    ct_texts = " ".join(ct_texts)

    ar_texts = articles[i].get_text()
    ar_texts.append(articles[i].get_title())
    ar_texts = " ".join(ar_texts)
        
    jd = [server.get_jds(ct_texts), server.get_jds(ar_texts)]
    jds.append(jd)
    st = [server.get_sts(ct_texts), server.get_sts(ar_texts)]
    sts.append(st)

In [10]:
server.close_server()

In [ ]:
max_precision = 0

from dataframe import calculate_attributes

for i in range(1,100):
    jds_similarities = []
    for j in range(len(jds)):
        jds_similarities.append(calculate_attributes.get_jds_sts_basic_similarities(jds[j][0][:i],jds[j][1][:i]))
    
    df['jds_similarity'] = jds_similarities

    # normalize
    df['namespace_size'] = common_functions.normalize(df['namespace_size'])
    df['last_name_length'] = common_functions.normalize(df['last_name_length'])
    df['year_difference'] = common_functions.normalize(df['year_difference'])

    # save csv
    df_to_save = df.drop(['CT', 'PubMed', 'ct_last_name', 'ct_first_name_initial', 'ct_first_name',
          'ct_organization', 'ct_mail', 'ct_year', 'ct_initials', 'ct_title', 'ct_country', 'ct_city', 
            'ar_last_name','ar_first_name_initial','ar_first_name','ar_organization', 'ar_mail', 'ar_initials',
                  'ar_year', 'ar_title', 'ar_location'], axis = 1)
    csv = df_to_save.to_csv(index=False)
    file = codecs.open("dataframe.csv", "w", "utf-8")
    file.write(csv)
    file.close()

    # model
    df_model = pd.read_csv('dataframe.csv', encoding = 'utf-8')
    x = df_model.drop('common_answer', axis=1)
    y = df_model['common_answer']

    avg_precision = 0
    for k in range(100):
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.33, random_state = k)
        classifier = RandomForest(0)
        classifier.create_model(x_train, y_train)
        pred = classifier.predict(x_test)
        avg_precision += precision_recall_fscore_support(y_test, pred, average='weighted')[0]
    avg_precision /= 100

    print("i:",i,"average precision:",avg_precision)

    if avg_precision > max_precision:
        max_precision = avg_precision
        best_i = i
        
print(best_i)

In [ ]:
max_precision = 0

for i in range(1,100):
    jds_similarities = []
    for j in range(len(jds)):
        jds_similarities.append(calculate_attributes.get_jds_sts_percentage_similarities(jds[j][0][:i],jds[j][1][:i]))
    
    df['jds_similarity'] = jds_similarities

    # normalize
    df['namespace_size'] = common_functions.normalize(df['namespace_size'])
    df['last_name_length'] = common_functions.normalize(df['last_name_length'])
    df['year_difference'] = common_functions.normalize(df['year_difference'])

    # save csv
    df_to_save = df.drop(['CT', 'PubMed', 'ct_last_name', 'ct_first_name_initial', 'ct_first_name',
          'ct_organization', 'ct_mail', 'ct_year', 'ct_initials', 'ct_title', 'ct_country', 'ct_city', 
            'ar_last_name','ar_first_name_initial','ar_first_name','ar_organization', 'ar_mail', 'ar_initials',
                  'ar_year', 'ar_title', 'ar_location'], axis = 1)
    csv = df_to_save.to_csv(index=False)
    file = codecs.open("dataframe.csv", "w", "utf-8")
    file.write(csv)
    file.close()

    # model
    df_model = pd.read_csv('dataframe.csv', encoding = 'utf-8')
    x = df_model.drop('common_answer', axis=1)
    y = df_model['common_answer']

    avg_precision = 0
    for k in range(100):
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.33, random_state = k)
        classifier = RandomForest(0)
        classifier.create_model(x_train, y_train)
        pred = classifier.predict(x_test)
        avg_precision += precision_recall_fscore_support(y_test, pred, average='weighted')[0]
    avg_precision /= 100

    print("i:",i,"average precision:",avg_precision)

    if avg_precision > max_precision:
        max_precision = avg_precision
        best_i = i
        
print(best_i)

In [ ]:
max_precision = 0

for i in range(1,100):
    jds_similarities = []
    for j in range(len(jds)):
        jds_similarities.append(calculate_attributes.get_jds_sts_ranking_similarities(jds[j][0][:i],jds[j][1][:i]))
    
    df['jds_similarity'] = jds_similarities

    # normalize
    df['namespace_size'] = common_functions.normalize(df['namespace_size'])
    df['last_name_length'] = common_functions.normalize(df['last_name_length'])
    df['year_difference'] = common_functions.normalize(df['year_difference'])

    # save csv
    df_to_save = df.drop(['CT', 'PubMed', 'ct_last_name', 'ct_first_name_initial', 'ct_first_name',
          'ct_organization', 'ct_mail', 'ct_year', 'ct_initials', 'ct_title', 'ct_country', 'ct_city', 
            'ar_last_name','ar_first_name_initial','ar_first_name','ar_organization', 'ar_mail', 'ar_initials',
                  'ar_year', 'ar_title', 'ar_location'], axis = 1)
    csv = df_to_save.to_csv(index=False)
    file = codecs.open("dataframe.csv", "w", "utf-8")
    file.write(csv)
    file.close()

    # model
    df_model = pd.read_csv('dataframe.csv', encoding = 'utf-8')
    x = df_model.drop('common_answer', axis=1)
    y = df_model['common_answer']

    avg_precision = 0
    for k in range(100):
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.33, random_state = k)
        classifier = RandomForest(0)
        classifier.create_model(x_train, y_train)
        pred = classifier.predict(x_test)
        avg_precision += precision_recall_fscore_support(y_test, pred, average='weighted')[0]
    avg_precision /= 100

    print("i:",i,"average precision:",avg_precision)

    if avg_precision > max_precision:
        max_precision = avg_precision
        best_i = i
        
print(best_i)

In [ ]:
max_precision = 0

from dataframe import calculate_attributes

for i in range(1,100):
    jds_similarities = []
    for j in range(len(jds)):
        jds_similarities.append(calculate_attributes.get_jds_sts_percentage_ranking_similarities(jds[j][0][:i],jds[j][1][:i]))
    
    df['jds_similarity'] = jds_similarities

    # normalize
    df['namespace_size'] = common_functions.normalize(df['namespace_size'])
    df['last_name_length'] = common_functions.normalize(df['last_name_length'])
    df['year_difference'] = common_functions.normalize(df['year_difference'])

    # save csv
    df_to_save = df.drop(['CT', 'PubMed', 'ct_last_name', 'ct_first_name_initial', 'ct_first_name',
          'ct_organization', 'ct_mail', 'ct_year', 'ct_initials', 'ct_title', 'ct_country', 'ct_city', 
            'ar_last_name','ar_first_name_initial','ar_first_name','ar_organization', 'ar_mail', 'ar_initials',
                  'ar_year', 'ar_title', 'ar_location'], axis = 1)
    csv = df_to_save.to_csv(index=False)
    file = codecs.open("dataframe.csv", "w", "utf-8")
    file.write(csv)
    file.close()

    # model
    df_model = pd.read_csv('dataframe.csv', encoding = 'utf-8')
    x = df_model.drop('common_answer', axis=1)
    y = df_model['common_answer']

    avg_precision = 0
    for k in range(100):
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.33, random_state = k)
        classifier = RandomForest(0)
        classifier.create_model(x_train, y_train)
        pred = classifier.predict(x_test)
        avg_precision += precision_recall_fscore_support(y_test, pred, average='weighted')[0]
    avg_precision /= 100

    print("i:",i,"average precision:",avg_precision)

    if avg_precision > max_precision:
        max_precision = avg_precision
        best_i = i
        
print(best_i)

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:25333)
Traceback (most recent call last):
  File "C:\Users\Brescia\Anaconda3\lib\site-packages\py4j\java_gateway.py", line 1174, in send_command
    self.socket.sendall(command.encode("utf-8"))
ConnectionResetError: [WinError 10054] Connessione in corso interrotta forzatamente dall'host remoto

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Brescia\Anaconda3\lib\site-packages\py4j\java_gateway.py", line 1014, in send_command
    response = connection.send_command(command)
  File "C:\Users\Brescia\Anaconda3\lib\site-packages\py4j\java_gateway.py", line 1178, in send_command
    "Error while sending", e, proto.ERROR_ON_SEND)
py4j.protocol.Py4JNetworkError: Error while sending

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Brescia\Anaconda3\lib\sit

i: 1 average precision: 0.8623513050483576
i: 2 average precision: 0.874671198599299
i: 3 average precision: 0.8789202433581603
i: 4 average precision: 0.8866703757224255
i: 5 average precision: 0.8855148504985801
i: 6 average precision: 0.8824169682413325
i: 7 average precision: 0.8838600744674077
i: 8 average precision: 0.8850843584594454
i: 9 average precision: 0.8841165118750487
i: 10 average precision: 0.882348198786051
i: 11 average precision: 0.8824125661405143


ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:25333)
Traceback (most recent call last):
  File "C:\Users\Brescia\Anaconda3\lib\site-packages\py4j\java_gateway.py", line 958, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Brescia\Anaconda3\lib\site-packages\py4j\java_gateway.py", line 1096, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [WinError 10061] Impossibile stabilire la connessione. Rifiuto persistente del computer di destinazione
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:25333)
Traceback (most recent call last):
  File "C:\Users\Brescia\Anaconda3\lib\site-packages\py4j\java_gateway.py", line 958, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty 

ConnectionRefusedError: [WinError 10061] Impossibile stabilire la connessione. Rifiuto persistente del computer di destinazione
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:25333)
Traceback (most recent call last):
  File "C:\Users\Brescia\Anaconda3\lib\site-packages\py4j\java_gateway.py", line 958, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Brescia\Anaconda3\lib\site-packages\py4j\java_gateway.py", line 1096, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [WinError 10061] Impossibile stabilire la connessione. Rifiuto persistente del computer di destinazione
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:25333)
Traceback (most recent call last):
  File "C:\Users\Brescia\Anaconda3\lib\si

ConnectionRefusedError: [WinError 10061] Impossibile stabilire la connessione. Rifiuto persistente del computer di destinazione
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:25333)
Traceback (most recent call last):
  File "C:\Users\Brescia\Anaconda3\lib\site-packages\py4j\java_gateway.py", line 958, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Brescia\Anaconda3\lib\site-packages\py4j\java_gateway.py", line 1096, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [WinError 10061] Impossibile stabilire la connessione. Rifiuto persistente del computer di destinazione
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:25333)
Traceback (most recent call last):
  File "C:\Users\Brescia\Anaconda3\lib\si

ConnectionRefusedError: [WinError 10061] Impossibile stabilire la connessione. Rifiuto persistente del computer di destinazione
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:25333)
Traceback (most recent call last):
  File "C:\Users\Brescia\Anaconda3\lib\site-packages\py4j\java_gateway.py", line 958, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Brescia\Anaconda3\lib\site-packages\py4j\java_gateway.py", line 1096, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [WinError 10061] Impossibile stabilire la connessione. Rifiuto persistente del computer di destinazione
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:25333)
Traceback (most recent call last):
  File "C:\Users\Brescia\Anaconda3\lib\si

ConnectionRefusedError: [WinError 10061] Impossibile stabilire la connessione. Rifiuto persistente del computer di destinazione
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:25333)
Traceback (most recent call last):
  File "C:\Users\Brescia\Anaconda3\lib\site-packages\py4j\java_gateway.py", line 958, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Brescia\Anaconda3\lib\site-packages\py4j\java_gateway.py", line 1096, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [WinError 10061] Impossibile stabilire la connessione. Rifiuto persistente del computer di destinazione
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:25333)
Traceback (most recent call last):
  File "C:\Users\Brescia\Anaconda3\lib\si

i: 12 average precision: 0.883428996204982
i: 13 average precision: 0.883540513853113
i: 14 average precision: 0.8845211354813904
i: 15 average precision: 0.8826406244777452
i: 16 average precision: 0.8836838972618187
i: 17 average precision: 0.8846376273930099
i: 18 average precision: 0.8853256606645179
i: 19 average precision: 0.8836691603615514
i: 20 average precision: 0.8839230891474892
i: 21 average precision: 0.8837149769086229
i: 22 average precision: 0.8824835544109534
i: 23 average precision: 0.8836308339291334
i: 24 average precision: 0.8846986084699897
i: 25 average precision: 0.8842409671903079
i: 26 average precision: 0.8850878969324191
i: 27 average precision: 0.8845085613130125
i: 28 average precision: 0.8843521838564993
i: 29 average precision: 0.8845018554585435
i: 30 average precision: 0.8843597853665054
i: 31 average precision: 0.8845773165288178
i: 32 average precision: 0.8846851013306761
i: 33 average precision: 0.8852900619937588
i: 34 average precision: 0.8854669

In [ ]:
importlib.reload(article_info)
importlib.reload(ct_info)
importlib.reload(calculate_attributes)
importlib.reload(clinical_trial)
importlib.reload(common_functions)

In [ ]:
df